## ⚙️ 1. Setup: Install Libraries

First, let's install the specific version of the Google Agent Development Kit (ADK) that this notebook is built with. Pinning the version ensures our code will always work as expected.

In [ ]:
!pip install google-adk==1.13.0 -q

## 🔑 2. Authentication: Configure Your API Key

Next, we need to securely provide our Google API key. This code will create a secure input prompt for you to paste your key. It then sets the key as an environment variable, which is the standard way the ADK authenticates your requests.

In [ ]:
import os
from getpass import getpass

# Prompt the user for their API key securely
api_key = getpass('Enter your Google API Key: ')


# Set the API key as an environment variable for ADK to use
os.environ['GOOGLE_API_KEY'] = api_key

print("✅ API Key configured successfully! Let the fun begin.")

## 🛠️ 3. Create a State-Updating Tool

To make our workflow more intelligent, we first need a custom tool that can save extracted information into the session's memory. This function takes an event_type and city and saves them to the tool_context.state.



## 📥 4. Create the "Intake" Agent

This agent acts as the front door to our workflow. Its only job is to take the user's natural language query (e.g., "Find vendors for a corporate launch party in Austin"), extract the key parameters, and then call our update_session_state tool to save them.

In [ ]:
from google.adk.agents import Agent

## 🧑‍🍳 5. Create Specialist Agent 1: The Catering Scout

This is our first specialist agent for the parallel workflow. Its only job is to search for catering options, using the {{event_type}} and {{city}} variables that were saved to the session state by the intake_agent.

In [ ]:
from google.adk.agents import Agent
from google.adk.tools import google_search

## 🎤 6. Create Specialist Agent 2: The Entertainment Scout

This is our second specialist agent. Its only job is to search for entertainment options. This task is completely independent of finding a caterer, which makes it a perfect candidate for parallel execution.

## ⚡ 7. Build the Parallel Workflow

Here, we use ParallelAgent to create a workflow that runs our two scout agents at the same time. The ADK will execute both searches concurrently, which can significantly speed up information gathering.

## 🧑‍🎨 8. Create the Synthesizer Agent

After the parallel searches are complete, we need an agent to combine the results into a single, coherent response. The vendor_coordinator_agent's job is to take the catering_options and entertainment_options and present them in an organized list.

## ➡️ 9. Assemble the Full "Fan-Out, Fan-In" Workflow

This is our final, complete workflow. We create a SequentialAgent that orchestrates the entire process: first, the intake_agent parses the query; second, the parallel_vendor_search "fans out" to find vendors; and third, the vendor_coordinator_agent "fans in" to synthesize the results.

## 🚀 10. Build the Execution Engine

This is our helper function for running queries, unchanged from previous articles. It handles the core ADK logic of initializing the Runner and streaming events. The state is now managed by our intake_agent, so no state_delta is needed here.

In [ ]:
from IPython.display import display, Markdown

from google.adk.sessions import Session
from google.genai.types import Content, Part
from google.adk.runners import Runner

async def run_agent_query(agent: Agent, query: str, session: Session, user_id: str):
    """Initializes a runner and executes a query for a given agent and session."""
    print(f"\n🚀 Running query for agent: '{agent.name}' in session: '{session.id}'...")

    runner = Runner(
        agent=agent,
        session_service=session_service,
        app_name=agent.name
    )

    final_response = ""
    try:
        async for event in runner.run_async(
            user_id=user_id,
            session_id=session.id,
            new_message=Content(parts=[Part(text=query)], role="user")
        ):
            if event.is_final_response():
                final_response = event.content.parts[0].text
    except Exception as e:
        final_response = f"An error occurred: {e}"


    print("\n" + "-"*50)
    print("✅ Final Response:")
    display(Markdown(final_response))
    print("-"*50 + "\n")

    return final_response

## ✨ 11. Initialize Session and Run the Workflow

Finally, we set up our InMemorySessionService and our main execution block. We create a session and then call our full_parallel_workflow with a single, natural-language query. The intake agent will handle extracting the parameters automatically.

In [ ]:
from google.adk.sessions import InMemorySessionService

# --- Initialize our Session Service ---
# This one service will manage all the different sessions in our notebook.
session_service = InMemorySessionService()
user_id = "adk_event_planner_001"

In [ ]:
async def run_stateful_orchestrator():
  session = await session_service.create_session(
        app_name=full_parallel_workflow.name,
        user_id=user_id
  )
  query = "Find vendors for a corporate launch party in Austin."
  print(f"User: {query}\n")
  await run_agent_query(full_parallel_workflow, query, session, user_id)

# Run the full system
await run_stateful_orchestrator()